In [6]:
import numpy as np

def Skew(w):
    skew_mat = np.array([[0, -w[2], w[1]],
                        [w[2], 0, -w[0]],
                        [-w[1], w[0], 0]], dtype = object)
    return skew_mat

def matrix_exp(w_hat, theta):
    skew_w = Skew(w_hat)
    R = np.eye(3) + np.sin(theta) * skew_w + (1 - np.cos(theta)) * np.matmul(skew_w, skew_w)
    return R

def screw_exp(screw, theta):
    w_hat = screw[:3]
    v = screw[3:]

    w_hat_skew = Skew(w_hat)

    R = matrix_exp(w_hat, theta)
    G_theta = np.eye(3) * theta + (1 - np.cos(theta)) * w_hat_skew + (theta - np.sin(theta)) * np.matmul(w_hat_skew, w_hat_skew)

    T = np.vstack((np.hstack((R, np.matmul(G_theta, v.reshape((-1, 1))))),
                       [0, 0, 0, 1]))
    return T
    
def Ad(T):
    p = T[:3, 3]
    R = T[:3, :3]
    p_mat = Skew(p)
    Adjoint = np.vstack((np.hstack((R, np.zeros((3, 3)))),
                             np.hstack((p_mat @ R, R))))
    return Adjoint

In [13]:
import numpy as np
import time

from scipy.linalg import logm

w = np.array([[0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1]])
q = np.array([[0, 0, 0], [0, 0, 0.1253], [0, 0.0834, 0.1253], [0, 0.1668, 0.1253], [0, 0.1668, 0.0753]])
v = -np.cross(w,q)
s1 = np.vstack([w[0].reshape(-1,1), v[0].reshape(-1,1)])
s2 = np.vstack([w[1].reshape(-1,1), v[1].reshape(-1,1)])
s3 = np.vstack([w[2].reshape(-1,1), v[2].reshape(-1,1)])
s4 = np.vstack([w[3].reshape(-1,1), v[3].reshape(-1,1)])
s5 = np.vstack([w[4].reshape(-1,1), v[4].reshape(-1,1)])

q_end = np.array([[0], [0.1668], [0.0479]])
M = np.vstack([np.hstack([np.eye(3), q_end]),
                  np.array([0, 0, 0, 1])])

# Initial Pos = [0; 0.12; 0.0665]
th = np.deg2rad([[0.0315], [52.0915], [-86.5612], [34.4697], [-0.0315]])

pos_d = np.array([[0.017, 0.12, 0.0665], [0.035, 0.12, 0.0665], [0.055, 0.12, 0.0665],
                  [0.075, 0.12, 0.0665], [0.055, 0.12, 0.0665], [0.035, 0.12, 0.0665],
                  [0.017, 0.12, 0.0665], [0, 0.12, 0.0665], [-0.017, 0.12, 0.0665],
                  [-0.035, 0.12, 0.0665], [-0.055, 0.12, 0.0665], [-0.075, 0.12, 0.0665],
                  [-0.055, 0.12, 0.0665], [-0.035, 0.12, 0.0665], [-0.017, 0.12, 0.0665],
                  [0, 0.12, 0.0665]])

for j in range(16):
    x_d = pos_d[j].reshape(-1,1)
    T_sd = np.vstack([np.hstack([np.eye(3), x_d]), [0, 0, 0, 1]])
    
    k = 1
    dt = 1.2

    w_b = np.array([[1], [1], [1]])
    v_b = np.array([[1], [1], [1]])
    
    while (np.linalg.norm(w_b) > 0.001 or np.linalg.norm(v_b) > 0.0001):

        # Forward Kinematic
        exp1 = screw_exp(s1, th[0])
        exp2 = screw_exp(s2, th[1])
        exp3 = screw_exp(s3, th[2])
        exp4 = screw_exp(s4, th[3])
        exp5 = screw_exp(s5, th[4])
        T_sb = exp1 @ exp2 @ exp3 @ exp4 @ exp5 @ M 
        T_sb = T_sb.astype(dtype = 'float64')

        # Inverse Kinematic
        T_bs = np.linalg.inv(T_sb)
        AdT_bs = Ad(T_bs)

        Js1 = s1
        Js2 = Ad(exp1) @ s2
        Js3 = Ad(exp1 @ exp2) @ s3
        Js4 = Ad(exp1 @ exp2 @ exp3) @ s4
        Js5 = Ad(exp1 @ exp2 @ exp3 @ exp4) @ s5
        Js = np.hstack([Js1, Js2, Js3, Js4, Js5])
        Js = Js.astype(dtype = 'float64')
        Jb = AdT_bs @ Js
        Jb = Jb.astype(dtype = 'float64')

        T_bd = T_bs @ T_sd
        V_bmat = logm(T_bd) * 0.07
        w_b = np.array([V_bmat[2, 1], V_bmat[0, 2], V_bmat[1, 0]])
        w_b = w_b.reshape(-1,1)
        v_b = V_bmat[:3, 3]
        v_b = v_b.reshape(-1,1)
        V_b = np.vstack([w_b, v_b])

        dth = np.linalg.pinv(Jb) @ V_b
        th = th + dth * dt
        th_deg = np.rad2deg(th)

        thd = np.array([th_deg[0]+90, th_deg[1], th_deg[2]+90, th_deg[3], -th_deg[4]+90])
        thd = thd.reshape(1,-1)
        
        k += 1

        if k == 250:
            break
            
    j += 1

In [14]:
th_deg

array([[  0.57507225],
       [ 52.09189266],
       [-86.56199626],
       [ 34.4701036 ],
       [ -0.57507225]])